In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from timeit import default_timer as timer  
from numba import jit, cuda

In [2]:
df = pd.read_csv(r'us_accidents_weather_data.csv')
data = df[['ID', 'Severity', 'Weather_Timestamp', 'Temperature(F)',
                        'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
                        'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition',
                        'Sunrise_Sunset']]
k = 3
eps = 13
iterationsMax = 20
clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
c = np.full(len(data), -1)

In [6]:
# data: points df
# clusters: clusters df
@jit(target_backend='cuda', nogil=True, cache=True) 
def initClusters(data, clusters):
    for i, ii in enumerate(clusters.iloc[0]):
        clmax = data.max().iloc[i]
        clmin = data.min().iloc[i]
        for j,jj in clusters.iterrows():
            clusters.iloc[j].iloc[i] = (clmax - clmin) * np.random.random() + clmin

initClusters(data, clusters)
print(clusters)

C:\Users\justi\AppData\Local\Temp\ipykernel_4868\3457171471.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\AppData\Local\Temp\ipykernel_4868\3457171471.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "initClusters" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_4868\3457171471.py (3)

File "..\..\..\..\AppData\Local\Temp\ipykernel_4868\3457171471.py", line 3:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda', nogil=True, cache=True)
C:\Users\justi\A

   petallength  petalwidth  sepallength  sepalwidth
0     6.195748    0.761539     6.822444    4.388768
1     1.140577    2.352503     4.978926    3.021417
2     5.585646    1.185892     4.488814    4.155491


In [4]:
# p: index of point 
# c: index of cluster
# clusters: clusters df
# data: points df
@jit(target_backend='cuda', nogil=True, cache=True) 
def sse(p: int, c: int, clusters: pd.DataFrame, data: pd.DataFrame) -> float:
    error: float = 0.0
    for i, ii in enumerate(clusters):
        error += (clusters.iloc[c][i] - data.iloc[p][i])**2
    return error


sse(0,0,clusters,data)


C:\Users\justi\AppData\Local\Temp\ipykernel_4868\2120149909.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_4868\2120149909.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sse" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_4868\2120149909.py (5)

File "..\..\..\..\AppData\Local\Temp\ipykernel_4868\2120149909.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_4868\2120149909.py:5: NumbaWa

23.28672026844218

In [27]:

# clusters: clusters df
# data: points df
# c: the cluster assignment array
@jit(target_backend='cuda') 
def reassignPoints(data, clusters, c):
    for i,ii in data.iterrows():
        bestIndex = -1
        bestSSE = 100000000
        for j,jj in clusters.iterrows():
            currentSSE = sse(i, j, clusters, data)
            if(currentSSE < bestSSE):
                bestSSE = currentSSE
                bestIndex = j
        c[i] = bestIndex

reassignPoints(data, clusters, c)

C:\Users\justi\AppData\Local\Temp\ipykernel_3188\68328681.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\68328681.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "reassignPoints" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_3188\68328681.py (4)

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\68328681.py", line 4:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\68328681.py:4: NumbaW

In [28]:
# clusters: clusters df
# data: points df
# c: the cluster assignment array
@jit(target_backend='cuda') 
def totalError(c, clusters, data):
    error = 0.0
    for i in range(len(c)):
        error = error + sse(i, c[i], clusters, data)
    return(error)

totalError(c, clusters, data)

C:\Users\justi\AppData\Local\Temp\ipykernel_3188\1850685268.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\1850685268.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "totalError" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_3188\1850685268.py (4)

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\1850685268.py", line 4:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\1850685268.py:4: 

505.42933861856795

In [29]:
@jit(target_backend='cuda') 
def clusterContent(c, t):
    total = 0
    for i in range(len(c)):
        if (c[i] == t):
            total = total + 1
    return (total)

clusterContent(c, 0)


C:\Users\justi\AppData\Local\Temp\ipykernel_3188\845319178.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


54

In [30]:
# clusters: clusters df
# data: points df
# c: the cluster assignment array
# cluster: the focal cluster
@jit(target_backend='cuda') 
def clusterError(c, cluster, clusters, data):
    error = 0.0
    for i in range(len(c)):
        if(c[i] == cluster):
            error = error + sse(i, c[i], clusters, data)
    return(error)

clusterError(c, 0, clusters, data)

C:\Users\justi\AppData\Local\Temp\ipykernel_3188\4154215476.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\4154215476.py:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "clusterError" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_3188\4154215476.py (5)

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\4154215476.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\4154215476.py:5

207.88816842414104

In [31]:
@jit(target_backend='cuda') 
def moveClusters(c, clusters, data):
    for i,ii in clusters.iterrows():
        for j,jj in enumerate(clusters.iloc[i]):
            sum = 0.0
            totP = 0.0
            for k in range(len(c)):
                if (c[k] == i):
                    sum = sum + data.iloc[k][j]
                    totP = totP + 1.0
            if (totP != 0):
                clusters.iloc[i][j] = sum/totP

print(clusters)
moveClusters(c, clusters, data)
print(clusters)

C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2903512841.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2903512841.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "moveClusters" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2903512841.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\2903512841.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')


   petallength  petalwidth  sepallength  sepalwidth
0     1.934645    1.550807     4.992855    2.095712
1     6.741669    1.582714     7.566291    4.348865
2     3.961026    0.214806     6.101115    3.337629


C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2903512841.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "moveClusters" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\2903512841.py", line 3:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
c:\Python39\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "moveClusters" was compiled in object mode without forceobj=True, but has lifted loops.

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\2903512841.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
c:\Python39\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is deprecated behaviour that w

   petallength  petalwidth  sepallength  sepalwidth
0     1.598148    0.301852     5.005556    3.335185
1     5.846875    2.131250     6.912500    3.100000
2     4.537500    1.489063     6.015625    2.793750


In [32]:
@jit(target_backend='cuda') 
def KMeans(data, k, eps, iterationsMax):
    clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
    c = np.full(len(data), -1)
    initClusters(data, clusters)

    oldError = -1 * eps
    newError = 0

    for a in range(1,iterationsMax):
        reassignPoints(data, clusters, c)
        moveClusters(c, clusters, data)
        newError = totalError(c, clusters, data)
        #print(newError)
        if(abs(newError - oldError) < eps):
            break
        oldError = newError
    #print(newError)
    return(newError)
KMeans(data, k, eps, iterationsMax)

C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2033785141.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2033785141.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "KMeans" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2033785141.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\2033785141.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2033785141.py:1: Numb

85.23257582417583

In [33]:
@jit(target_backend='cuda') 
def KMeans2(data, k, eps, iterationsMax):
    clusters = pd.DataFrame(data = 0.0, columns=data.columns, index=range(k))
    c = np.full(len(data), -1)
    initClusters(data, clusters)

    oldError = -1 * eps
    newError = 0

    for a in range(1,iterationsMax):
        reassignPoints(data, clusters, c)
        moveClusters(c, clusters, data)
        newError = totalError(c, clusters, data)
        #print(newError)
        if(abs(newError - oldError) < eps):
            break
        oldError = newError
    #print(newError)
    output = clusters
    cle = np.full(k,-1.0)
    ccn = np.full(k, -1)
    for i in range(k):
        cle[i] = round(clusterError(c, i, clusters, data),3)
        ccn[i] = clusterContent(c, i)
    output['error'] = cle
    output['numOfPoints'] = ccn

    return(output)

start = timer()
print(KMeans2(data, k, eps, iterationsMax))
print("without GPU:", timer()-start)   

C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2472246041.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2472246041.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "KMeans2" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2472246041.py (1)

File "..\..\..\..\AppData\Local\Temp\ipykernel_3188\2472246041.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\justi\AppData\Local\Temp\ipykernel_3188\2472246041.py:1: Num

   petallength  petalwidth  sepallength  sepalwidth   error  numOfPoints
0     1.494118    0.260784     5.007843    3.400000  19.107           51
1     4.485075    1.467164     5.968657    2.768657  44.201           67
2     5.846875    2.131250     6.912500    3.100000  18.703           32
without GPU: 15.644001599999996
